In [ ]:
import pandas as pd

# Load dimensi
dim_product = pd.read_csv("dim_product.csv")
dim_site = pd.read_csv("dim_site.csv")
dim_date = pd.read_csv("dim_date.csv")

# Load transaksi per struk
transactions = pd.read_csv("mock_transactions_per_struk.csv")

# Join transactions dengan dim_product
transactions = transactions.merge(
    dim_product[["PRODUCT_KEY", "PRODUCT_NAME"]],
    on="PRODUCT_NAME",
    how="left"
)

# Join transactions dengan dim_site
transactions = transactions.merge(
    dim_site[["SITE_ID", "SITE_DESC"]],
    left_on="STORE_NAME",
    right_on="SITE_DESC",
    how="left"
)

# Join transactions dengan dim_date
transactions["TRANSACTION_DATE"] = pd.to_datetime(transactions["TRANSACTION_DATE"])
dim_date["PERIOD_DESC"] = pd.to_datetime(dim_date["PERIOD_DESC"])
transactions = transactions.merge(
    dim_date[["PERIOD_KEY", "PERIOD_DESC"]],
    left_on="TRANSACTION_DATE",
    right_on="PERIOD_DESC",
    how="left"
)

# Fact transaction header
fact_transaction_header = transactions.groupby(
    ["TRANSACTION_ID", "PERIOD_KEY", "SITE_ID"]
).agg(TOTAL_PRICE=("ITEM_TOTAL_PRICE", "sum")).reset_index()

# Fact transaction details
fact_transaction_detail = transactions[[
    "TRANSACTION_ID",
    "PRODUCT_KEY",
    "GRAIN",
    "ITEM_TOTAL_PRICE"
]]

# Save ke file CSV
fact_transaction_header.to_csv("fact_transaction_header.csv", index=False)
fact_transaction_detail.to_csv("fact_transaction_detail.csv", index=False)

print("Fact transactions saved to 'fact_transaction_header.csv' and 'fact_transaction_detail.csv'.")